In [13]:
import json, os
from util import json_write, json_read

In [14]:
database_folder = 'Kanji Database'

kanji_2000_fname = 'kanji_2000.json'
course_mazzi_fname = 'Course_mazzi.json'
course_jdict_fname = 'Course_jdict.json'
course_kanji_data_jellyfish_fname = 'Course_kanji_data_jellyfish.json'

course_jdict_clean_fname = 'Course_jdict_clean.json'
course_jdict_clean_fname = 'Course_database.json'
kanji_214_fname = 'kanji_214.json'

course_mazzi = json_read(os.path.join(database_folder, course_mazzi_fname))
course_jdict = json_read(os.path.join(database_folder, course_jdict_fname))
course_kanji_data_jellyfish = json_read(os.path.join(database_folder, course_kanji_data_jellyfish_fname))

In [15]:
def update_kanji_char(similar, opt=1):
    kanji_2000_data = json_read(os.path.join(database_folder, kanji_2000_fname))
    char_data = kanji_2000_data['kanji_2000'].keys()
    new_similar = []
    for char in similar:
        if opt == 1:
            if char in char_data:
                new_similar.append(f"{char}({kanji_2000_data['kanji_2000'][char]['vn_sound'].upper()})")
            else:
                new_similar.append(f"{char}")
        else:
            if char in char_data and char not in new_similar:
                new_similar.append(f"{char}")
    new_similar = ''.join(new_similar)
    return new_similar

In [16]:

course = 'Course'
input_data = course_jdict[course]
course_data = {}

for lesson, word_list in input_data.items():
    new_lesson_data = []
    for word_data in word_list: 
        index = word_data['word_index']
        word = word_data['word']

        pronoun_clean = []
        for pronoun in word_data['pronoun_data']:
            pronoun_clean.append('###'.join(pronoun.split('###')[1:]))
        
        [vn_sound, kanji_214, stroke, kunyomi, onyomi, components] =  pronoun_clean

        components = components.split('###')
        cpn_clean = []
        kanji_214_db = json_read(os.path.join(database_folder, kanji_214_fname))
        for cpn in components:
            check = False
            for key in kanji_214_db['kanji_214']:
                if cpn == key['key']:
                    check = True
                    break    
                else: check = False

            if check: cpn_clean.append(f"[{cpn}({key['vn_sound'].upper()}) - {key['mean']}]")
            else: cpn_clean.append(f"[{cpn}]")

        components = ' + '.join(cpn_clean)

        comments = word_data['comment_data']
        comments = [cmt.replace('\n', '') for cmt in comments]
        comments = [f" - {cmt}" for cmt in comments]
        comments = '\n'.join(comments)

        similar_kanjis = word_data['kanji_similar_data']
        similar_kanjis = [kj.split('###') for kj in similar_kanjis]
        similar_kanjis = [f" - {kj[0]}({kj[1]})\n" for kj in similar_kanjis]
        similar_kanjis = ''.join(similar_kanjis)

        related_kanjis = word_data['related_word_data']
        related_kanjis = [kj.split('###') for kj in related_kanjis]
        related_kanjis = [f" - {kj[0]}: {kj[1]}\n" for kj in related_kanjis]
        related_kanjis = ''.join(related_kanjis)

        vn_sound = course_mazzi[course][lesson][index-1]['vn_sound']
        word_mean = course_mazzi[course][lesson][index-1]['word_mean']
        word_image_svg_code = course_mazzi[course][lesson][index-1]['word_image_svg_code']

        onyomi_example = course_kanji_data_jellyfish[course][lesson][index-1]['onyomi']
        onyomi_example = onyomi_example.replace(':', ': ')
    
        kunyomi_example = course_kanji_data_jellyfish[course][lesson][index-1]['kunyomi']
        kunyomi_example = kunyomi_example.replace(':', ': ')

        onyomi = update_kanji_char(onyomi)
        onyomi_example = update_kanji_char(onyomi_example)
        kunyomi = update_kanji_char(kunyomi)
        kunyomi_example = update_kanji_char(kunyomi_example)
        # components = update_kanji_char(components)
        comments = update_kanji_char(comments)
        similar_kanjis = update_kanji_char(similar_kanjis)
        related_kanjis = update_kanji_char(related_kanjis)
        word_mean = update_kanji_char(word_mean)

        search_keyword =  update_kanji_char(onyomi_example, 0) + update_kanji_char(kunyomi_example, 0)
        kanji_jdict_url = f"https://jdict.net/search?keyword={word}{search_keyword}&type=kanji"
        kanji_mazzi_url = f"https://vi.mazii.net/vi-VN/search/kanji/javi/{word}{search_keyword}"

        new_lesson_data.append({
            'lesson': lesson,
            'index': str(index),
            'word': word,
            'vn_sound': vn_sound,
            'kanji_214': kanji_214,
            'stroke': stroke,
            'onyomi': onyomi,
            'onyomi_example': onyomi_example,
            'kunyomi': kunyomi,
            'kunyomi_example': kunyomi_example,
            'components': components,
            'comments': comments,
            'similar_kanjis': similar_kanjis,
            'related_kanjis': related_kanjis,
            'word_mean': word_mean,
            'word_image_svg_code': word_image_svg_code,
            'kanji_jdict_url': kanji_jdict_url,
            'kanji_mazzi_url': kanji_mazzi_url
        })

        course_data.update({
            f"{lesson}": new_lesson_data,
        })

    json_write(f'{database_folder}\\{course_jdict_clean_fname}', {
        f"{course}": course_data
    })